In [1]:
using LinearAlgebra
using Polyhedra

using CDDLib
using Clarabel, Ipopt

using TropicalFrechetMeans

In [2]:
function trop_normalize(x)
    return x .- first(x)
end

trop_normalize (generic function with 1 method)

# Simple example

Here, we define the sample. For this, we need to use `BigInt` as the underlying scalar type because some computations below overflow otherwise.

In [3]:
sample = broadcast.(big, [
    [2//10, 4//10, 2//1, 4//10, 2//1, 2//1],
    [2//1, 2//1, 2//1, 4//10, 4//10, 2//1],
    [4//10, 4//10, 2//1, 2//10, 2//1, 2//1]
])

3-element Vector{Vector{Rational{BigInt}}}:
 [1//5, 2//5, 2, 2//5, 2, 2]
 [2, 2, 2, 2//5, 2//5, 2]
 [2//5, 2//5, 2, 1//5, 2, 2]

## Clarabel

In [4]:
FM_clarabel = tropical_frechet_mean(sample; tol=0)

6-element Vector{Float64}:
 -0.7224697008896936
 -0.72246970056275
  0.08383364713211054
 -0.9980604959340645
 -0.1224618319156645
  0.08378590966924016

The following calculation suggests that for a tropical Fréchet mean the sum of square distances to the sample is close to $7.28$.

In [5]:
sos_clarabel = sum_of_trop_dist(FM_clarabel, sample; power=2) 

7.280000000839644500177929709347000404190869240814846689527169752073610092118919

We can also calculate the FM polytrope to obtain more Fréchet means.

In [6]:
P_clarabel = tropical_frechet_set(sample)

Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
10-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, 0, -1, 0, 0], 1//1)
 HalfSpace(Rational{BigInt}[1, 0, 0, 0, -1, 0], -3//5)
 HalfSpace(Rational{BigInt}[0, 1, -1, 0, 0, 0], -3//5)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, -1, 0], -3//5)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, -1], -3//5)
 HalfSpace(Rational{BigInt}[0, 0, 1, 0, -1, 0], 1//1)
 HalfSpace(Rational{BigInt}[0, 0, 0, 1, -1, 0], -4//5)
 HalfSpace(Rational{BigInt}[-1, 0, 0, 0, 1, 0], 3//5)
 HalfSpace(Rational{BigInt}[0, -1, 0, 0, 1, 0], 3//5)
 HalfSpace(Rational{BigInt}[0, 0, 0, 0, -1, 1], 1//1)

In [7]:
pts_clarabel = points(P_clarabel)

8-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[-3//5, -3//5, 0, -4//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 0, -8//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 1, -8//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 1, -4//5, 0, 0]
 Rational{BigInt}[-8//5, -8//5, -1, -13//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, -1, -9//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, 0, -13//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, 0, -9//5, -1, 0]

Using `Clarabel`, we obtain a FM polytrope $\overline{P}$ whose vertices all attain the same sum of square distances to the sample.

In [8]:
sum_of_trop_dist.(pts_clarabel, Ref(sample); power=2)

8-element Vector{Rational{BigInt}}:
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25

In particular, the vertices of $\overline{P}$ realize a sum of square distances of exactly $7.28$.

In [9]:
sum_of_trop_dist(pts_clarabel |> first, sample; power=2) |> Float64

7.28

## Ipopt

In [10]:
FM_ipopt = tropical_frechet_mean(Ipopt.Optimizer, sample; tol=0)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



6-element Vector{Float64}:
 -2.420568765584609
 -2.420568763105464
 -1.2271034242953822
 -2.9375139940978983
 -1.8205688128778803
 -1.227103331354988

The tropical FM we obtain from `Ipopt` beahves similarly to the one from `Clarabel`. 

In [11]:
sos_ipopt = sum_of_trop_dist(FM_ipopt, sample; power=2) 

7.280000004958296729356369774337790501729526162658762950850919841438990687265554

In [12]:
P_ipopt = tropical_frechet_set(Ipopt.Optimizer, sample);

In [13]:
pts_ipopt = points(P_ipopt)

8-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[-8//5, -8//5, -1, -9//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, 0, -9//5, -1, 0]
 Rational{BigInt}[-3//5, -3//5, 0, -4//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 1, -4//5, 0, 0]
 Rational{BigInt}[-8//5, -8//5, -1, -13//5, -1, 0]
 Rational{BigInt}[-8//5, -8//5, 0, -13//5, -1, 0]
 Rational{BigInt}[-3//5, -3//5, 0, -8//5, 0, 0]
 Rational{BigInt}[-3//5, -3//5, 1, -8//5, 0, 0]

In [14]:
sum_of_trop_dist.(pts_ipopt, Ref(sample); power=2)

8-element Vector{Rational{BigInt}}:
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25
 182//25

# Random instances

In [15]:
D = [10,15,20]
N = [1,2,3]
meta_sample_size = 3

function perform_random_experiment()
    return map(D) do d
        map(N) do n
            map(1:meta_sample_size) do _
                k = d*n
                rand_sample = [rand(big.(1:20), d) for _ in 1:k]

                num_FM = tropical_frechet_mean(rand_sample; tol=0)
                P = tropical_frechet_set(rand_sample)

                num_c = sum_of_trop_dist(num_FM, rand_sample)
                P_c = map(points(P)) do pt
                    sum_of_trop_dist(num_FM, rand_sample)
                end

                num_FM, num_c, P, P_c
            end
        end
    end
end

perform_random_experiment (generic function with 1 method)

In [16]:
import Random
Random.seed!(0xACE0FBA2E)

@time results = perform_random_experiment();

 66.055506 seconds (459.73 M allocations: 23.800 GiB, 21.56% gc time, 1.77% compilation time)


Checking that the FM polytrope for each random sample agrees with Theorem 9.

In [17]:
consistency = map(enumerate(D)) do (i,d)
    map(enumerate(N)) do (j,n)
        map(1:meta_sample_size) do k
            ((unique(results[i][j][k][4]) |> length) == 1),i,j,k
        end
    end
end |> splat(vcat) |> splat(vcat)

27-element Vector{Tuple{Bool, Int64, Int64, Int64}}:
 (1, 1, 1, 1)
 (1, 1, 1, 2)
 (1, 1, 1, 3)
 (1, 1, 2, 1)
 (1, 1, 2, 2)
 (1, 1, 2, 3)
 (1, 1, 3, 1)
 (1, 1, 3, 2)
 (1, 1, 3, 3)
 (1, 2, 1, 1)
 (1, 2, 1, 2)
 (1, 2, 1, 3)
 (1, 2, 2, 1)
 ⋮
 (0, 2, 3, 1)
 (1, 2, 3, 2)
 (1, 2, 3, 3)
 (1, 3, 1, 1)
 (1, 3, 1, 2)
 (1, 3, 1, 3)
 (1, 3, 2, 1)
 (1, 3, 2, 2)
 (1, 3, 2, 3)
 (0, 3, 3, 1)
 (1, 3, 3, 2)
 (1, 3, 3, 3)

In [18]:
filter(!first, consistency)

2-element Vector{Tuple{Bool, Int64, Int64, Int64}}:
 (0, 2, 3, 1)
 (0, 3, 3, 1)

For the random instances which do not agree with Theorem 9, it turns out that the computation of the FM polytrope yielded an empty set.

In [19]:
map(filter(!first, consistency)) do (_,i,j,k)
    isempty(results[i][j][k][3])
end

2-element Vector{Bool}:
 1
 1

For those random sample for which the computed FM polytrope is non-empty, we can verify that the numerical tropical FM and the vertices of the FM polytrope have almost equal sum of square distances. Inspecting the sum of square distances for the numerical FMs also suggests that the error due to the floating-point computations is in the order of $10^{-5}$.

In [20]:
map(filter(first, consistency)) do (_,i,j,k)
    num_c = results[i][j][k][2]
    P_c = unique(results[i][j][k][4]) |> first
    
    num_c - P_c, num_c
end

25-element Vector{Tuple{BigFloat, BigFloat}}:
 (0.0, 2364.00000213302925379833252316082871643536968944475972949210970819745435828052)
 (0.0, 2548.500000144574807013252527193276421079771055227132047129114249472430398131551)
 (0.0, 2175.000000252998916528238477970410584581769123865835346975927677704021334648132)
 (0.0, 4972.500000329768312815697242786293112024650539834770738845127721344406470510568)
 (0.0, 5064.000000904541727576025562192173721974797690368612818883270962558719278934216)
 (0.0, 4892.500000245949827411519075327337498015565069872260356065338243103981499468325)
 (0.0, 7566.000000352803835601224566145271599891250946956012007649002404743084765348371)
 (0.0, 7855.000001098983865705323911591579471815131830922385169913013669429346919059753)
 (0.0, 8458.000001576282939875933121259005707075842580384647286008675629313726535007945)
 (0.0, 4564.500000308576697775773861168543776568854921277267328439761334980626506906826)
 (0.0, 4785.00000014436352913465992905174479592118335834995439170